In [ ]:
import os
from typing import Optional

import torch
from dotenv import load_dotenv
from duckdb import duckdb, DuckDBPyConnection
from duckdb.typing import DuckDBPyType
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, Document
from llama_index.core.node_parser import SentenceSplitter

load_dotenv()

hf_token = os.getenv("HF_TOKEN")
db_path = '../data/llama.db'
documents_path = '/Users/ada.sh/Developments/Projects/TechnicalDocsAssistant/tests/test_data/textual'

In [ ]:
## Data type for
ARRAY_TYPE = DuckDBPyType(list[float])

## Token size of each chunk
CHUNK_SIZE = 1024

# Token size of overlap between chunks
CHUNK_OVERLAP = 200

# HuggingFace identifier for embedding model
MODEL_NAME = 'BAAI/bge-small-en-v1.5'

In [ ]:



def close_db(db):
    if db is not None:
        db.close()
    else:
        print('You must provide the connection to the database')

def open_db(db_path: str = None, in_memory: Optional[bool] = False) -> DuckDBPyConnection:
  if db_path is None or in_memory:
    return duckdb.connect(':memory:')
  return duckdb.connect(db_path)


def load_extension(conn: DuckDBPyConnection, extension: str = 'vss') -> DuckDBPyConnection:
    # Load any extension into duckdb
    # For RAG, we're loading Vector Similarity Search
  try:
    conn.install_extension(extension)
    conn.load_extension(extension)
    conn.execute("SET GLOBAL hnsw_enable_experimental_persistence = true;")
  except Exception as e:
    print(f"Error loading extension: {e}")
  return conn


def initialize_schema(conn: DuckDBPyConnection) -> None:
  # Create documents table
  conn.execute(
    f"""
        CREATE TABLE IF NOT EXISTS documents (
            id INT PRIMARY KEY,
            file TEXT,
            text TEXT
        );
    """
    )

  # Create chunks table w/ relation to documents
  conn.execute("""
    CREATE TABLE IF NOT EXISTS chunks (
    id INT PRIMARY KEY,
    doc_id INT,
    chunk_text TEXT,
    chunk_index INT,
    FOREIGN KEY(doc_id) REFERENCES documents(id)
    );
  """)

  # Create embeddings table w/ relation to chunks
  conn.execute(
    f"""
        CREATE TABLE IF NOT EXISTS embeddings (
            chunk_id INT,
            embedding {ARRAY_TYPE},
            FOREIGN KEY (chunk_id) REFERENCES chunks(id)
        );
    """
    )

# We will split the documents into 1024 tokens with 200 token overlap
#
# Sentence splitter prefers not to split sentences or paragraphs, where
# possible.
def setup_node_parser() -> SentenceSplitter:

    return SentenceSplitter(
        chunk_size=CHUNK_SIZE,
        chunk_overlap=CHUNK_OVERLAP,
        separator=" ",
        paragraph_separator="\n\n"
    )

# Insert docs into db and then process them into chunks
def process_documents(documents: list[Document], db: DuckDBPyConnection, node_parser: SentenceSplitter) -> None:

    for i, doc in enumerate(documents):
        db.execute(
            """
            INSERT INTO documents (id, file, text)
            VALUES (?, ?, ?)""",
            [i, doc.metadata.get('file_path', ''), doc.text]
        )

        nodes = node_parser.get_nodes_from_documents([doc])

        for chunk_idx, node in enumerate(nodes):
            # The hacky id incrementer here should be replaced with a sequence
            db.execute(
                """
                INSERT INTO chunks (id, doc_id, chunk_text, chunk_index)
                VALUES (( SELECT COALESCE(MAX(id), -1) + 1 FROM chunks), ?, ?, ?)""",
                [i, node.text, chunk_idx]
            )

# Basic embedding model setup
def setup_embedding_model() -> HuggingFaceEmbedding:
  return HuggingFaceEmbedding(
    model_name=MODEL_NAME,
      device='cuda' if torch.cuda.is_available() else 'cpu',
  )

# Generate embeddings from chunks and store them
def create_embeddings(db: DuckDBPyConnection, embedding_model: HuggingFaceEmbedding) -> None:

    chunks = db.execute("""
    SELECT id, chunk_text FROM chunks""").fetchall()

    for idx, chunk in enumerate(chunks):
        chunk_id, chunk_text = chunk
        embedding = embedding_model.get_text_embedding(chunk_text)

        db.execute("""
        INSERT INTO embeddings (chunk_id, embedding)
        VALUES (?, ?)""",
                   [chunk_id, embedding])

# Compare embedding of prompt against all embeddings.
# This uses array_inner_product for normalized vectors
# and does not yet implement an HNSW index
def query_documentation(
        query_text: str,
        db: DuckDBPyConnection,
        embed_model: HuggingFaceEmbedding,
        top_k: int = 5,
        similarity_threshold: float = 0.0,
) -> list[dict]:

    query_embedding = embed_model.get_text_embedding(query_text)

    results = db.execute("""
      WITH top_matches AS (
        SELECT
          chunk_id,
          array_inner_product(
          embedding::FLOAT[384], ?::FLOAT[384]) as similarity,
          FROM embeddings
          ORDER BY similarity DESC
          LIMIT ?
      )
      SELECT c.id as chunk_id,
      c.chunk_text,
      c.chunk_index,
      c.doc_id as doc_id,
      d.file,
      d.text as full_doc_text,
      m.similarity
      from top_matches m
      JOIN chunks c on c.id = m.chunk_id
      JOIN documents d on d.id = c.doc_id
      WHERE m.similarity >= ?
      ORDER BY m.similarity DESC
    """, [query_embedding, top_k, similarity_threshold]).fetchall()

    return [
      {
          "chunk_id": row[0],
          "chunk_text": row[1],
          "chunk_index": row[2],
          "doc_id": row[3],
          "file": row[4],
          "document_text": row[5],
          "similarity": row[6]

      }
                  for row in results
    ]


In [ ]:
# Initializes database connection. If in_memory is
# set to True, duckdb will use an ephemeral in memory
# database. If False and passed a file path, it will
# create or open a local file-based db.
db = open_db(db_path, in_memory=False)

load_extension(db, extension='vss')
initialize_schema(db)

node_parser = setup_node_parser()
embed_model = setup_embedding_model()


In [ ]:
# Set the extensions of the files we're interested in
required_extensions = ['.md']

# Loads files recursively from a directory. SimpleDirectoryReader
# will load the correct file readers for the files being loaded
reader = SimpleDirectoryReader(documents_path, required_exts=required_extensions, recursive=True)
documents = reader.load_data()


In [ ]:
process_documents(documents, db=db, node_parser=node_parser)

In [ ]:
create_embeddings(db, embed_model)


In [ ]:

# Prompt our database with the question or request that will
# be passed to the LLM. We create an embedding, search for
# embeddings in the same proximity in vector space,
# and return the top five.
#
# We take the doc ids to pull the right docs to pass
results = query_documentation(
    "How do I create a new app in Textualize?",
    top_k=5,
    db=db,
    embed_model=embed_model,
    similarity_threshold=0.7,
)

for r in results:
    print(f"\nSimilarity: {r['similarity']:.4f}")
    print(f"File: {r['file']}")
    print(f"Chunk {r['chunk_index']}:")
    print(f"Text: {r['chunk_text']}")
    print("---")

docs_to_send = set([r['doc_id'] for r in results])

In [ ]:
def query_docs(db: DuckDBPyConnection, ids: list[int]) -> list[str]:

    results = db.execute("""
    SELECT id, file, text FROM documents WHERE id IN ?""", [ids]).fetchall()

    return results

In [ ]:
docs = query_docs(db, list(docs_to_send))

In [ ]:
db.close()